In [4]:
from pandas.io.json import json_normalize 
import requests, json 
import pandas as pd
import time
from datetime import datetime 



In [25]:
# This function gathers Bitcoin price history from Bitmex API, only 1m, 5m, 1h and 1d timeframes are aivable
# End date format is yyyy-mm-dd, but function will gather more data because API gives data with bins size 750

def getDataBitmex(binSize,end_date):
    ' Gather data from Bitmex API, binSize = "1m" ,"5m", "1h" or "1d", End date format yyyy-mm-dd'
    
    datastart=requests.get(f'https://www.bitmex.com/api/v1/trade/bucketed?binSize={binSize}&partial=false&symbol=XBTUSD&count=750&reverse=true').json()
    time_object=datastart[-1]['timestamp']
    del datastart[-1]
    while True:
        time.sleep(1.05)
        addresults=requests.get(f'https://www.bitmex.com/api/v1/trade/bucketed?binSize={binSize}&partial=false&symbol=XBTUSD&count=750&reverse=true&endTime={time_object}').json()
        datastart.extend(addresults)
        del datastart[-1]
        time_object=datastart[-1]['timestamp']
        print(time_object)
        if datetime.strptime(time_object[:10],'%Y-%m-%d')<datetime.strptime(end_date,'%Y-%m-%d'):
            break
    return datastart

In [26]:
# choose time frame and end date
price_history=getDataBitmex('5m','2019-10-10')

2019-10-15T15:25:00.000Z
2019-10-13T01:05:00.000Z
2019-10-10T10:45:00.000Z
2019-10-07T20:25:00.000Z


In [19]:
#save results from json to pandas dataframe
btc5m=json_normalize(price_history)

In [20]:
btc5m.head()

,close,foreignNotional,high,homeNotional,lastSize,low,open,symbol,timestamp,trades,turnover,volume,vwap
0,8258.0,61325703,8285.0,7418.201397,300,8243.0,8243.5,XBTUSD,2019-10-20T20:05:00.000Z,12143,741820139689,61325703,8267.1958
1,8243.5,15316975,8245.0,1858.333280,2500,8234.5,8235.0,XBTUSD,2019-10-20T20:00:00.000Z,3658,185833328033,15316975,8242.6640
2,8235.0,9040543,8236.0,1098.877527,1030,8220.5,8223.5,XBTUSD,2019-10-20T19:55:00.000Z,2393,109887752740,9040543,8227.7440
3,8223.5,20392501,8243.5,2478.390535,137,8217.0,8240.0,XBTUSD,2019-10-20T19:50:00.000Z,4749,247839053488,20392501,8228.4210
4,8240.0,20359725,8240.0,2473.725501,17581,8216.0,8216.5,XBTUSD,2019-10-20T19:45:00.000Z,4787,247372550117,20359725,8230.4527


In [22]:
# change timestamp to index and leave only neccessary columns, this thay only high, low and close are needed
btc5m.index=btc5m['timestamp']
btc5m=btc5m[['high','low','close']]

In [24]:
btc5m.head()

,high,low,close
timestamp,,,
2019-10-20T20:05:00.000Z,8285.0,8243.0,8258.0
2019-10-20T20:00:00.000Z,8245.0,8234.5,8243.5
2019-10-20T19:55:00.000Z,8236.0,8220.5,8235.0
2019-10-20T19:50:00.000Z,8243.5,8217.0,8223.5
2019-10-20T19:45:00.000Z,8240.0,8216.0,8240.0


In [8]:
#save data
btc5m.to_csv('BTC_history_5m.csv')